This code creates interpolation data for incidents between the specified
start_date and end_date, and saves it as csv file in the folder:
/fromIncidentToInterpolationData/incidentInterpolationData

In [68]:

import matplotlib.pyplot as plt
import pandas as pd
import json
from sqlalchemy import create_engine

Presets, agreed upon for all diagrams

In [69]:
x_incident = 3000
t_incident = 60
time_recording_before_incident = 60
time_recording_after_incident = 420 #7hours

# the filename for the speedflow data 
file_name = "/Users/vree/Documents/UTwente/Masters UTwente/Data Science/DM Project/Speed-flow/Maart/intensiteit-snelheid-rotterdam-15-21-Maart/intensiteit-snelheid-rotterdam-15-21-Maart.csv"



presets for current run

In [70]:
# filter dates between 15th of March 2020 and 21th of March 2020
start_date = '2020-03-15'
end_date = '2020-03-21'

In [71]:
driver='postgresql'
username='dab_ds23241a_223'
dbname=username # it is the same as the username
password='Bh6KFgcWazMMCPvZ'
server='bronto.ewi.utwente.nl'
port='5432'

# Create an engine instance
alchemy_engine = create_engine(f'{driver}://{username}:{password}@{server}:{port}/{dbname}')

# Connect to PostgreSQL server
db_connection = alchemy_engine.connect()

# Read the data into a_df
a_df = pd.read_sql(f"select * from \"project\".\"Incidents_With_Cameras\"", db_connection)

# Check if 'before' and 'after' columns are already dictionaries
if a_df['before'].apply(type).eq(dict).all() and a_df['after'].apply(type).eq(dict).all():
    print('Columns are already dictionaries.')
else:
    # Convert the 'before' and 'after' columns from JSON strings back to dictionaries if they are not already
    a_df['before'] = a_df['before'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
    a_df['after'] = a_df['after'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
# Close the database connection
db_connection.close()
print(a_df.shape)
a_df.head()


(266, 9)


,id,starttijd,type,gedetailleerd_type,primaire_locatie_lengtegraad,primaire_locatie_breedtegraad,segment,before,after
0,RWS02_0000157188_157188,2020-03-01 08:38:28,vehicle_obstruction,emergencyVehicle,4.366420,51.920952,north,{'RWS01_MONIBAS_0041hrl0706ra': 0.022954588858...,{'GEO0K_K_RWSTI357132': 0.16579609461072695}
1,RWS02_0000157197_157197,2020-03-01 10:00:57,vehicle_obstruction,emergencyVehicle,4.432840,51.928864,west-east,{'RWS01_MONIBAS_0201hrr0284ra': 0.094239550154...,{'RWS01_MONIBAS_0201hrr0285ra': 0.065785586736...
2,RWS02_0000157269_157269,2020-03-01 13:21:58,vehicle_obstruction,emergencyVehicle,4.456025,51.864422,east-west,{'RWS01_MONIBAS_0150vwy0557ra': 0.249505010285...,{'RWS01_MONIBAS_0150vwy0553ra': 0.229508095798...
3,RWS03_816372_1,2020-03-01 15:19:57,vehicle_obstruction,brokenDownVehicle,4.538568,51.909523,south,{'RWS01_MONIBAS_0160vwx0193ra': 0.896763594374...,{'RWS01_MONIBAS_0160vwx0202ra': 0.001663796286...
4,RWS02_0000157393_157393,2020-03-01 23:03:29,vehicle_obstruction,emergencyVehicle,4.439620,51.868599,east-west,{'RWS01_MONIBAS_0150vwy0544ra': 0.166980200322...,{'RWS01_MONIBAS_0150vwy0540ra': 0.210197772623...


Filter between the preset dates

In [72]:
# filter between preset dates: start_date and end_date
a_df = a_df[(a_df['starttijd'] >= start_date) & (a_df['starttijd'] <= end_date)]
print(a_df.shape)

(13, 9)


In [73]:
# returns cameras and their true x for an incident
#  so a list of camera ids and their x values
def create_x_variable_for_incident_cameras(row):
    locations_before = row['before']
    locations_after = row['after']
    
    # create dataframe with distance before, where the row entry is a json with a key
    # for camera id and the value is the distance
    before_cameras_data = []
    
    before_cameras_data = [{'camera_id': key, 'x': x_incident - (value * 1000)} for key, value in locations_before.items()]
    
    before_cameras_x = pd.DataFrame(before_cameras_data)
    
    # create dataframe with distance after, where the row entry is a json with a key
    # for camera id and the value is the distance
    after_cameras_data = []
    
    after_cameras_data = [{'camera_id': key, 'x': x_incident + (value * 1000)} for key, value in locations_after.items()]
        
    after_cameras_x = pd.DataFrame(after_cameras_data)
    df = pd.concat([before_cameras_x, after_cameras_x])
    print(df.head(3))
    return df


 
    
    
    
    
    

In [74]:
# times must be delivered as pd.datetime
def speed_flow_data_for_incident(camera_ids, start_time, end_time):
    # read speed flow data in chunks of 10000 and filter on camera id and time
    # only keep start_meetperiode, id_meetlocatie, gem_snelheid
    
    
    chunksize = 10000
    chunks = []
    for chunk in pd.read_csv(file_name, chunksize=chunksize):
        chunk = chunk[(chunk['id_meetlocatie'].isin(camera_ids)) & (pd.to_datetime(chunk['start_meetperiode']) >= start_time) & (pd.to_datetime(chunk['start_meetperiode']) <= end_time)]
        chunk = chunk[['start_meetperiode', 'id_meetlocatie', 'gem_snelheid']]
        chunks.append(chunk)
    return pd.concat(chunks)


Creating the incident interpolation data

In [75]:
# incident_row = a_df.iloc[0]

# cameras_x_df = create_x_variable_for_incident_cameras(incident_row)
    
# camera_ids = cameras_x_df['camera_id'].tolist()

# incident_time = pd.to_datetime(incident_row['starttijd'])
# start_time_for_diagram = incident_time - pd.Timedelta(minutes=time_recording_before_incident)
# end_time_for_diagram = incident_time + pd.Timedelta(minutes=time_recording_after_incident)
# # returns df with start_meetperiode, id_meetlocatie, gem_snelheid
# speedflow_data = speed_flow_data_for_incident(camera_ids, start_time_for_diagram, end_time_for_diagram)

# # let the start_meetperiode be the time in minutes since the incident
# speedflow_data['start_meetperiode'] = (pd.to_datetime(speedflow_data['start_meetperiode']) - incident_time).dt.total_seconds() / 60.0

# merged_data = pd.merge(cameras_x_df, speedflow_data, left_on='camera_id', right_on='id_meetlocatie')

# # rename columns: x->x, gem_snelheid->v, start_meetperiode->t
# merged_data = merged_data.rename(columns={'x': 'x', 'gem_snelheid': 'v', 'start_meetperiode': 't'})


In [76]:
# aggregate duplicates where x and t are the same, and take the average of v
def aggregate_data(data):
    return data.groupby(['x', 't']).agg({'v': 'mean'}).reset_index()



In [77]:
# merged_data.head()

In [78]:
# retrieving a list of cameras and their x values
def create_interpolation_data_for_incident(incident_row):
    

    cameras_x_df = create_x_variable_for_incident_cameras(incident_row)
        
    camera_ids = cameras_x_df['camera_id'].tolist()

    incident_time = pd.to_datetime(incident_row['starttijd'])
    start_time_for_diagram = incident_time - pd.Timedelta(minutes=time_recording_before_incident)
    end_time_for_diagram = incident_time + pd.Timedelta(minutes=time_recording_after_incident)
    # returns df with start_meetperiode, id_meetlocatie, gem_snelheid
    speedflow_data = speed_flow_data_for_incident(camera_ids, start_time_for_diagram, end_time_for_diagram)

    print("speed flow data for incident was: ", speedflow_data.head())

    # let the start_meetperiode be the time in minutes since the incident
    speedflow_data['start_meetperiode'] = ((pd.to_datetime(speedflow_data['start_meetperiode']) - incident_time).dt.total_seconds() / 60.0)+t_incident

    merged_data = pd.merge(cameras_x_df, speedflow_data, left_on='camera_id', right_on='id_meetlocatie')

    # rename columns: x->x, gem_snelheid->v, start_meetperiode->t
    merged_data = merged_data.rename(columns={'x': 'x', 'gem_snelheid': 'v', 'start_meetperiode': 't'})
    
    return merged_data


incident_row = a_df.iloc[0]
incident_data = create_interpolation_data_for_incident(incident_row)



    
        
    
    




                     camera_id            x
0  RWS01_MONIBAS_0201hrr0324ra  2799.197115
1  RWS01_MONIBAS_0201hrr0322ra  2545.692751
2  RWS01_MONIBAS_0201hrr0319ra  2237.016259


/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_43207/252438380.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk = chunk[(chunk['id_meetlocatie'].isin(camera_ids)) & (pd.to_datetime(chunk['start_meetperiode']) >= start_time) & (pd.to_datetime(chunk['start_meetperiode']) <= end_time)]
/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_43207/252438380.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk = chunk[(chunk['id_meetlocatie'].isin(camera_ids)) & (pd.to_datetime(chunk['start_meetperiode']) >= start_time) & (pd.to_datetime(chunk['start_meetperiode']) <= end_time)]
/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_43207/252438380.py:10: UserWarning: Could not inf

speed flow data for incident was:              start_meetperiode               id_meetlocatie  gem_snelheid
54483752  2020-03-15 13:50:00  RWS01_MONIBAS_0201hrr0309ra          79.0
54483753  2020-03-15 13:50:00  RWS01_MONIBAS_0201hrr0309ra          76.0
54483754  2020-03-15 13:50:00  RWS01_MONIBAS_0201hrr0309ra          73.0
54483755  2020-03-15 13:51:00  RWS01_MONIBAS_0201hrr0309ra          80.0
54483756  2020-03-15 13:51:00  RWS01_MONIBAS_0201hrr0309ra          76.0


In [81]:
# only keep the columns x, v, t
incident_data = incident_data[['x', 'v', 't']]

# there's multiple measurements within a minute, which lead to duplicates for x, t . 
incident_data = aggregate_data(incident_data)
# show number of rows incident data
print('number of rows incident data: ', len(incident_data))

incident_data_remove_negatives = incident_data[incident_data['v'] >= 0]
# show number of rows after removing negatives
print('number of rows after removing negatives; ', len(incident_data_remove_negatives))


# write the data to a csv file in ./incidentInterpolationData/, use the incident id as filename
incident_data_remove_negatives.to_csv(f'./incidentInterpolationData/{incident_row["id"]}.csv', index=False)

incident_data_remove_negatives.head()

number of rows incident data:  8640
number of rows after removing negatives;  8633


,x,t,v
0,1324.576342,0.533333,76.000000
1,1324.576342,1.533333,76.333333
2,1324.576342,2.533333,78.333333
3,1324.576342,3.533333,76.333333
4,1324.576342,4.533333,78.666667


In [ ]:
print('before removing duplicates: ', incident_data.shape)

# remove duplicates for x and t 
no_duplicates = incident_data.drop_duplicates(subset=['x', 't'])

print('after removing duplicates: ', no_duplicates.shape)


before removing duplicates:  (8640, 3)
after removing duplicates:  (8640, 3)
